## Convert ESO results

Idea is to convert to raw ESO results into the same format than the outputs from OCF ML models

### Load ESO file

Load the ESO file

### Load ESO GSP metadata

Load the ESO GSP metadata. We will need to convert between GSP ID and GSP name, and know the location of each GSP

### Load test dataset 

Let load the test dataset meta file. This is so we know which times and what locations / gsps to compare

### Reduce ESO forecasts 

Reduce ESO forecast to the same as the test dataset

### Load Forecast Truth and Capacity

Load the GSP outturn value at each forecast value. Also load the capacity


In [1]:
from pathlib import Path

# location fo ESO forecasts
ESO_PV_FORECASTS_PATH = Path("/mnt/storage_b/data/ocf/solar_pv_nowcasting/other_organisations_pv_forecasts/National_Grid_ESO/NetCDF/ESO_GSP_PV_forecasts.nc")

# The locations of the tests dataset
TEST_DATASET_FILE ='/mnt/storage_ssd_4tb/data/ocf/solar_pv_nowcasting/nowcasting_dataset_pipeline/prepared_ML_training_data/v15/test/spatial_and_temporal_locations_of_each_example.csv'

# The "ground truth" estimated total PV generation from each Grid Supply Point from Sheffield Solar:
GSP_ZARR_PATH = Path("/mnt/storage_b/data/ocf/solar_pv_nowcasting/nowcasting_dataset_pipeline/PV/GSP/v3/pv_gsp.zarr")

# Output csv
ESO_PV_FORECASTS_OUTPUT_FILE = Path("/mnt/storage_b/data/ocf/solar_pv_nowcasting/other_organisations_pv_forecasts/National_Grid_ESO/CSV/testset_v15.csv")



### Load ESO file

In [2]:
# load main file

import xarray as xr

eso_pv_forecasts_dataset = xr.open_dataset(ESO_PV_FORECASTS_PATH)
eso_pv_forecasts_dataset = eso_pv_forecasts_dataset.rename({'gsp_id': 'gsp_name'})
eso_pv_forecasts_dataset

<xarray.Dataset>
Dimensions:             (gsp_name: 350, step: 97, forecast_date_time: 9036)
Coordinates:
  * gsp_name            (gsp_name) object 'ABHA1' 'ABNE_P' ... 'WYMOM_1'
  * step                (step) timedelta64[ns] 00:00:00 ... 2 days 00:00:00
  * forecast_date_time  (forecast_date_time) datetime64[ns] 2020-06-01T00:30:...
Data variables:
    ASL                 (gsp_name, forecast_date_time, step) float32 ...
    ML                  (gsp_name, forecast_date_time, step) float32 ...

In [3]:
import pandas as pd

# Select just two timesteps: 30 minutes to 2 hours ahead:
ESO_FORECAST_ALGO_NAME = 'ASL'  # Either ASL or ML.
selected_eso_forecasts_dataarray = (
    eso_pv_forecasts_dataset[ESO_FORECAST_ALGO_NAME]
    .sel(step=slice(pd.Timedelta("30 minutes"), pd.Timedelta("2 hour"))))

selected_eso_forecasts_dataarray

<xarray.DataArray 'ASL' (gsp_name: 350, forecast_date_time: 9036, step: 4)>
[12650400 values with dtype=float32]
Coordinates:
  * gsp_name            (gsp_name) object 'ABHA1' 'ABNE_P' ... 'WYMOM_1'
  * step                (step) timedelta64[ns] 00:30:00 01:00:00 ... 02:00:00
  * forecast_date_time  (forecast_date_time) datetime64[ns] 2020-06-01T00:30:...

### Load ESO GSP metadata

In [4]:
from nowcasting_dataset.data_sources.gsp import eso
from nowcasting_dataset.geospatial import lat_lon_to_osgb

# Download metadata from ESO for each GSP.  We need this because
# the GSP PV data from Sheffield Solar uses integer gsp_ids, whilst
# the ESO forecasts use textual gsp_names.  So we need the metadata
# to allow us to map from gsp_id to gsp_name.
gsp_metadata = eso.get_gsp_metadata_from_eso(calculate_centroid=True)

gsp_id_to_name = (
    gsp_metadata[['gsp_id', 'gsp_name']]
    .dropna()
    .astype({'gsp_id': int})
    .set_index('gsp_id')
    .squeeze()
)

gsp_name_to_id = (
    gsp_metadata[['gsp_id', 'gsp_name']]
    .dropna()
    .astype({'gsp_name': str})
    .set_index('gsp_name')
    .squeeze()
)
print(gsp_id_to_name)

gsp_metadata["location_x"], gsp_metadata["location_y"] = lat_lon_to_osgb(
            lat=gsp_metadata["centroid_lat"], lon=gsp_metadata["centroid_lon"]
        )

print(gsp_metadata.columns)
print(gsp_metadata[['location_x','location_y']])



gsp_id
1      COWL_1
2      ECLA_1
3      ENDE_1
4      GREN_1
5      RATS_1
        ...  
334    MELK_1
335      KNAR
336    WYLF_1
337     DALM3
338      FINN
Name: gsp_name, Length: 338, dtype: object
Index(['Unnamed: 0', '_id', 'ng_id', 'ggd_id', 'gnode_id', 'gnode_name',
       'gnode_lat', 'gnode_lon', 'gsp_id', 'gsp_name', 'gsp_lat', 'gsp_lon',
       'dc_id', 'dc_name', 'dc_lat', 'dc_lon', 'region_id', 'region_name',
       'has_pv', 'pes_id', 'pes_name', 'RegionID', 'RegionName', 'centroid_x',
       'centroid_y', 'centroid_lat', 'centroid_lon', 'geometry', 'location_x',
       'location_y'],
      dtype='object')
        location_x     location_y
0    448889.644160  206989.365607
1    472005.739028  237635.701248
2    451521.981357  306026.426142
3    483798.618486  285794.816938
4    459869.284136  324199.443470
..             ...            ...
334  374787.393690  164683.411687
335  435628.316462  469958.011013
336  233135.276581  374042.059774
337  261505.755216  661584.85

### Load test dataset 

In [5]:
# get list of datetimes fof test set
import pandas as pd
import numpy as np
from tqdm import tqdm

# load location for test dataset
locations_df = pd.read_csv(TEST_DATASET_FILE)

# append gsp id to locations_df

# this causes allot of nans, so do it a loop
# new_df = pd.merge(locations_df, gsp_metadata[['gsp_id','location_x','location_y']],  
#                   how='left', left_on=['x_center_OSGB','y_center_OSGB'], 
#                   right_on = ['location_x','location_y'])

locations_df['gsp_id'] = 0.0
# loop over all the data points int he test set. Can take about 2 mins ~10,000 data points
for i in tqdm(range(len(locations_df))):
    x_meters_center = locations_df.x_center_OSGB[i]
    y_meters_center = locations_df.y_center_OSGB[i]
    
    meta_data_index = gsp_metadata.index[
            np.isclose(gsp_metadata.location_x, x_meters_center, rtol=1e-05, atol=1e-05)
            & np.isclose(gsp_metadata.location_y, y_meters_center, rtol=1e-05, atol=1e-05)
        ]
    gsp_id = gsp_metadata.loc[meta_data_index].gsp_id.values[0]
    locations_df.loc[i, 'gsp_id'] = gsp_id
    


100%|████████████████████████████████████| 12800/12800 [01:26<00:00, 148.03it/s]


### Reduce ESO forecasts 

In [6]:
# only select the 'forecast_date_time' from test set

# change them to have mins of 0 or 30, as ESO forecasts are at 0 or 30 mins. 
locations_df['t0_datetime_UTC_floor_30_mins'] = pd.to_datetime(locations_df['t0_datetime_UTC']).dt.floor('30T')
forecast_date_time = selected_eso_forecasts_dataarray.forecast_date_time

# make list of datetimes that are in 't0_datetime_utc' and 'forecast_date_time'
t0_datetime_utc = pd.DatetimeIndex(locations_df['t0_datetime_UTC_floor_30_mins']).unique()
forecast_date_time = pd.DatetimeIndex(forecast_date_time)
forecast_date_time = forecast_date_time.join(t0_datetime_utc, how='inner')

# select the eso forecasts that are in the test set

print(forecast_date_time.max())
print(forecast_date_time.min())
selected_eso_forecasts_dataarray = selected_eso_forecasts_dataarray.sel(forecast_date_time=forecast_date_time)

# also filter t0_datetimes
locations_df = locations_df[locations_df['t0_datetime_UTC_floor_30_mins'].isin(forecast_date_time)]
print(locations_df)


2021-08-31 15:00:00
2021-01-01 10:30:00
       Unnamed: 0      t0_datetime_UTC  x_center_OSGB  y_center_OSGB  gsp_id  \
0               0  2021-08-06 09:15:00  334280.694604  442872.263194   312.0   
3               3  2021-07-23 08:25:00  579801.863541  228093.718044    55.0   
4               4  2021-06-13 07:00:00  507636.659095  197928.890020    70.0   
8               8  2021-03-31 09:30:00  527405.340310  120108.039709    97.0   
13             13  2021-06-20 06:05:00  522189.093127  179628.309623    32.0   
...           ...                  ...            ...            ...     ...   
12793       12793  2021-06-01 08:20:00  461521.270673  340869.231097     7.0   
12794       12794  2021-06-29 16:00:00  483798.618486  285794.816938     4.0   
12796       12796  2021-05-11 08:00:00  536188.353965  153184.379039    20.0   
12797       12797  2021-05-23 11:20:00  451521.981357  306026.426142     3.0   
12799       12799  2021-06-01 16:05:00  266633.445256  713036.052822   274.0   


In [7]:
# Dont want to load any unnecessary data, so going to loop through t0_datetime_UTC and only save the forecast we need

eso_dataarrays_list =[]
# this can take about 2 seconds for ~5000 samples
for i in tqdm(range(len(locations_df))):
    t0_datetime_UTC_floor_30_mins = locations_df.t0_datetime_UTC_floor_30_mins.iloc[i]
    gsp_id = locations_df.gsp_id.iloc[i]
    gsp_name = gsp_id_to_name[gsp_id]
    
    
    
    one_eso_forecasts_dataarray = selected_eso_forecasts_dataarray.sel(forecast_date_time=t0_datetime_UTC_floor_30_mins)
    one_eso_forecasts_dataarray = one_eso_forecasts_dataarray.sel(gsp_name=gsp_name)
    
    eso_dataarrays_list.append(one_eso_forecasts_dataarray)
    
predictions = []
# this can take about 20 seconds for ~5000 samples
for eso_dataarrays in tqdm(eso_dataarrays_list):
    
    target_times = eso_dataarrays.forecast_date_time + eso_dataarrays.step
    forecast = eso_dataarrays.values
    gsp_id = gsp_name_to_id[eso_dataarrays.gsp_name.values]
    
    
    predictions_df = pd.DataFrame({'forecast_gsp_pv_outturn_mw':forecast,
                                   'target_datetime_utc':target_times})
    predictions_df['gsp_id'] = int(gsp_id)
    predictions_df['t0_datetime_utc'] = eso_dataarrays.forecast_date_time.values
    
    predictions.append(predictions_df)
    
    
predictions = pd.concat(predictions)
print(predictions)
    

    
    
    
    

100%|███████████████████████████████████████| 5422/5422 [02:00<00:00, 44.95it/s]


    forecast_gsp_pv_outturn_mw target_datetime_utc  gsp_id     t0_datetime_utc
0                     5.250000 2021-08-06 09:30:00     312 2021-08-06 09:00:00
1                     5.840000 2021-08-06 10:00:00     312 2021-08-06 09:00:00
2                     6.990000 2021-08-06 10:30:00     312 2021-08-06 09:00:00
3                     7.100000 2021-08-06 11:00:00     312 2021-08-06 09:00:00
0                    43.259998 2021-07-23 08:30:00      55 2021-07-23 08:00:00
..                         ...                 ...     ...                 ...
3                     4.480000 2021-05-23 13:00:00       3 2021-05-23 11:00:00
0                     0.150000 2021-06-01 16:30:00     274 2021-06-01 16:00:00
1                     0.120000 2021-06-01 17:00:00     274 2021-06-01 16:00:00
2                     0.070000 2021-06-01 17:30:00     274 2021-06-01 16:00:00
3                     0.050000 2021-06-01 18:00:00     274 2021-06-01 16:00:00

[21688 rows x 4 columns]


### Load Forecast Truth and Capacity

In [10]:
# now we need to find out the truth values
pv_live_dataset = xr.open_dataset(GSP_ZARR_PATH, engine="zarr")

# Convert 'gsp_id' from strings like '1', '2', etc. to ints
pv_live_dataset['gsp_id'] = pv_live_dataset['gsp_id'].astype(int)

predictions_and_truths = predictions
predictions_and_truths['actual_gsp_pv_outturn_mw'] = 0.0
predictions_and_truths['capacity_mwp'] = 0.0
# this can take about 1 min 30 seconds for ~5000 samples
for i in tqdm(range(len(predictions_and_truths))):
    
    target_datetime_utc = predictions_and_truths.target_datetime_utc.iloc[i]
    gsp_id = predictions_and_truths.gsp_id.iloc[i]
    
    one_pv_live_dataset = pv_live_dataset.sel(datetime_gmt=target_datetime_utc)
    one_pv_live_dataset = one_pv_live_dataset.sel(gsp_id=gsp_id)
    
    predictions_and_truths.actual_gsp_pv_outturn_mw.iloc[i] = one_pv_live_dataset.generation_mw.values
    predictions_and_truths.capacity_mwp.iloc[i] = one_pv_live_dataset.installedcapacity_mwp.values
    
print(predictions_and_truths)
    

# is genration mw normalized?
    
    
    




  0%|                                                 | 0/21688 [00:00<?, ?it/s]/home/peter/.local/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
100%|█████████████████████████████████████| 21688/21688 [07:07<00:00, 50.72it/s]


    forecast_gsp_pv_outturn_mw target_datetime_utc  gsp_id  \
0                     5.250000 2021-08-06 09:30:00     312   
1                     5.840000 2021-08-06 10:00:00     312   
2                     6.990000 2021-08-06 10:30:00     312   
3                     7.100000 2021-08-06 11:00:00     312   
0                    43.259998 2021-07-23 08:30:00      55   
..                         ...                 ...     ...   
3                     4.480000 2021-05-23 13:00:00       3   
0                     0.150000 2021-06-01 16:30:00     274   
1                     0.120000 2021-06-01 17:00:00     274   
2                     0.070000 2021-06-01 17:30:00     274   
3                     0.050000 2021-06-01 18:00:00     274   

       t0_datetime_utc  actual_gsp_pv_outturn_mw  capacity_mwp  
0  2021-08-06 09:00:00                  4.950830     38.756335  
1  2021-08-06 09:00:00                  9.729040     38.756335  
2  2021-08-06 09:00:00                 13.764800     38.7563

In [11]:
# save file to csv
predictions_and_truths.to_csv(ESO_PV_FORECASTS_OUTPUT_FILE)